In [ ]:
import pathlib

In [ ]:
import zipfile

In [ ]:
#Descompactando arquivo zipado:
with zipfile.ZipFile('/content/uvas.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/DataSets')

In [ ]:
#Caminho diretório DataSets:
data_dir = pathlib.Path('/content/DataSets/uvas')
data_dir

PosixPath('/content/DataSets/uvas')

In [ ]:
#Quantidade de imagens:
len(list(data_dir.glob('*/*.JPG')))

0

In [ ]:
#Verificando o número e nome de subfolders:
subfolders = [f.name for f in data_dir.iterdir() if f.is_dir()]
subfolders

FileNotFoundError: [Errno 2] No such file or directory: '/content/DataSets/uvas'

In [ ]:
import PIL

In [ ]:
#Criando lista de imagens do diretório LeafBlight:
leafblight = list(data_dir.glob('LeafBlight/*'))

In [ ]:
#Exibindo imagem das folhas em LeafBlight:
PIL.Image.open(str(leafblight[0]))

In [ ]:
import numpy as np

In [ ]:
#Verificando balanceamento do DataSet:

for subfolder in subfolders:
  #Criando o path de cada subfolder
  path = data_dir / subfolder
  #Criando lista de imagens do subfolder
  images = list(path.glob('*.JPG'))

  print(f'Classe {subfolder} tem {len(images)} imagens')

  #Verificando características da imagem
  if images:
    img = PIL.Image.open(str(images[0]))
    img_array = np.array(img)
    print(f'Dimensões da primeira imagem em {subfolder}: {img_array.shape} \n')

In [ ]:
import tensorflow as tf

In [ ]:
batch_size = 64
altura = 256
largura = 256

In [ ]:
#Separando conjunto de treino:
treino = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=568,
    image_size=(altura, largura),
    batch_size=batch_size
)

In [ ]:
#Separando conjunto de validação:
validacao = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=568,
    image_size=(altura, largura),
    batch_size=batch_size
)

In [ ]:
treino.class_names

In [ ]:
tf.random.set_seed(424242)

In [ ]:
#Treinando um modelo MLP sem CNN:
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(altura, largura, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [ ]:
#Compilando o modelo:
modelo.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
#Treinando o modelo:
epocas = 10

history = modelo.fit(
    treino,
    validation_data=validacao,
    epochs=epocas
)

In [ ]:
modelo.summary()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plota_resultados(history, epocas):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  intervalo_epocas = range(epocas)

  plt.figure(figsize=(12, 6))
  plt.subplot(1, 2, 1)
  plt.plot(intervalo_epocas, acc, 'r', label='Acurácia de Treino')
  plt.plot(intervalo_epocas, val_acc, 'b', label='Acurácia de Validação')
  plt.legend(loc='lower right')

  plt.subplot(1, 2, 2)
  plt.plot(intervalo_epocas, loss, 'r', label='Loss de Treino')
  plt.plot(intervalo_epocas, val_loss, 'b', label='Loss de Validação')
  plt.legend(loc='upper right')

  plt.show()

In [ ]:
plota_resultados(history,epocas)

In [ ]:
#Treinando um modelo MLP com CNN:
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(altura, largura, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [ ]:
modelo.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
epocas = 10

history = modelo.fit(
    treino,
    validation_data=validacao,
    epochs=epocas
)

In [ ]:
plota_resultados(history,epocas)

In [ ]:
modelo.summary()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>=0.93):
      print("\n Atingiu 93% de acurácia!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
#Treinando um modelo MLP com CNN e earlystopp:
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(altura, largura, 3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

modelo.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
epocas = 10

history = modelo.fit(
    treino,
    validation_data=validacao,
    epochs=epocas,
    callbacks=[callbacks]
)

In [ ]:
#Aumentando a variabilidade do DataSet:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.5),
    tf.keras.layers.RandomZoom(0.1)
])

In [ ]:
#Treinando um modelo MLP com CNN e earlystopp:
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(altura, largura, 3)),

    #Data augmentation:
    data_augmentation,

    #Normalizando pixels:
    tf.keras.layers.Rescaling(1./255),

    #Camadas Convolucionais:
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

modelo.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
epocas = 50

history = modelo.fit(
    treino,
    validation_data=validacao,
    epochs=epocas,
    callbacks=[callbacks]
)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 130s 6s/step - accuracy: 0.2689 - loss: 4.7070 - val_accuracy: 0.4313 - val_loss: 1.1812
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 139s 6s/step - accuracy: 0.4720 - loss: 1.0152 - val_accuracy: 0.6125 - val_loss: 0.8468
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 144s 6s/step - accuracy: 0.6657 - loss: 0.8287 - val_accuracy: 0.6906 - val_loss: 0.7092
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 138s 6s/step - accuracy: 0.7775 - loss: 0.5792 - val_accuracy: 0.6969 - val_loss: 0.8437
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 145s 6s/step - accuracy: 0.7853 - loss: 0.5013 - val_accuracy: 0.6250 - val_loss: 1.1348
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 140s 6s/step - accuracy: 0.8473 - loss: 0.3977 - val_accuracy: 0.7594 - val_loss: 0.6818
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 146s 6s/step - accuracy: 0.8327 - loss: 0.4322 - val_accuracy: 0.7219 - val_loss: 0.7293
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 137s 6s/step - accuracy: 0.8257 - loss: 0.4364 - val_accuracy: 0.7250 - v

In [ ]:
#Utilizando modelos pré-treinados:
modelo_base = tf.keras.applications.InceptionV3(
    input_shape=(altura, largura, 3),
    include_top=False,
    weights='imagenet'
)

In [ ]:
modelo_base.trainable = False

In [ ]:
modelo_base.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 127, 127, 32)   │            864 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 127, 127, 32)   │             96 │ conv2d_6[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 127, 127, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 125, 125, 32)   │          9,216 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 125, 125, 32)   │             96 │ conv2d_7[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 125, 125, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 125, 125, 64)   │         18,432 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 125, 125, 64)   │            192 │ conv2d_8[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 125, 125, 64)   │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_6           │ (None, 62, 62, 64)     │              0 │ activation_2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 62, 62, 80)     │          5,120 │ max_pooling2d_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 62, 62, 80)     │            240 │ conv2d_9[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 62, 62, 80)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_10 (Conv2D)        │ (None, 60, 60, 192)    │        138,240 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 60, 60, 192)    │            576 │ conv2d_10[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 21,802,784 (83.17 MB)